# Imports

In [ ]:
from pathlib import Path

import pandas as pd

import constants.column_names.dashboard as col_dashboard
import constants.column_names.silver as col_silver
from constants.paths import SILVER_YIELD_PATH

In [ ]:
SCENARII_DIR = Path("../models/baseline_xgboost_model/artifacts")
SCENARIO_126_PATH = SCENARII_DIR / "scenario_126_predictions.csv"
SCENARIO_245_PATH = SCENARII_DIR / "scenario_245_predictions.csv"
SCENARIO_585_PATH = SCENARII_DIR / "scenario_585_predictions.csv"

# Impact analysis

## Financial impact

To measure financial impact, we compare for a fixed area the price of production at average yield with the price of production by selecting departments to optimize yield (to do this, we assume we drop the $20$ departments with the lowest yield, and that we can relocate their production to the $69$ remaining departments).

The price of production is given by the formula:
$$
Price = Yield \times Area \times Barley\ Price 
$$

We take the barley price to be 120 $/ton which is the globa price of barley in January 2026 according to the [FRED](https://fred.stlouisfed.org/series/PBARLUSDM) database.

In [ ]:
BARLEY_PRICE = 120  # $/ton

In [ ]:
historical_yield_data = pd.read_parquet(SILVER_YIELD_PATH)

In [ ]:
avg_area_per_dep = historical_yield_data.groupby(col_silver.SILVER_NOM_DEP).agg(
    {col_silver.SILVER_AREA: "mean"}
)
avg_area_cultivated_per_dep = avg_area_per_dep.mean()

average_area_cultivated_per_year = (
    historical_yield_data.groupby(col_silver.SILVER_YEAR)
    .agg({col_silver.SILVER_AREA: "sum"})
    .mean()
)

number_of_deps_needed = (
    average_area_cultivated_per_year / avg_area_cultivated_per_dep
).values[0]

In [ ]:
scenario_126 = pd.read_csv(SCENARIO_126_PATH)
scenario_245 = pd.read_csv(SCENARIO_245_PATH)
scenario_585 = pd.read_csv(SCENARIO_585_PATH)

best_yield_126 = (
    scenario_126.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .nlargest(columns=col_dashboard.DASHBOARD_YIELD, n=70)
    .mean()
)
best_yield_245 = (
    scenario_245.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .nlargest(columns=col_dashboard.DASHBOARD_YIELD, n=70)
    .mean()
)
best_yield_585 = (
    scenario_585.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .nlargest(columns=col_dashboard.DASHBOARD_YIELD, n=70)
    .mean()
)

print(f"Best yield for scenario 126: {best_yield_126.values[0]:.2f} tons/ha")
print(f"Best yield for scenario 245: {best_yield_245.values[0]:.2f} tons/ha")
print(f"Best yield for scenario 585: {best_yield_585.values[0]:.2f} tons/ha")


average_yield_scenario_126 = (
    scenario_126.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .mean()
)
average_yield_scenario_245 = (
    scenario_245.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .mean()
)
average_yield_scenario_585 = (
    scenario_585.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .mean()
)

print(
    f"\nAverage yield for scenario 126: {average_yield_scenario_126.values[0]:.2f} "
    "tons/ha"
)
print(
    f"Average yield for scenario 245: {average_yield_scenario_245.values[0]:.2f} "
    "tons/ha"
)
print(
    f"Average yield for scenario 585: {average_yield_scenario_585.values[0]:.2f} "
    "tons/ha"
)

In [ ]:
financial_impact_scenario_126 = (
    (best_yield_126.values[0] - average_yield_scenario_126.values[0])
    * avg_area_cultivated_per_dep.values[0]
    * BARLEY_PRICE
) * number_of_deps_needed

financial_impact_scenario_245 = (
    (best_yield_245.values[0] - average_yield_scenario_245.values[0])
    * avg_area_cultivated_per_dep.values[0]
    * BARLEY_PRICE
) * number_of_deps_needed

financial_impact_scenario_585 = (
    (best_yield_585.values[0] - average_yield_scenario_585.values[0])
    * avg_area_cultivated_per_dep.values[0]
    * BARLEY_PRICE
) * number_of_deps_needed

print(f"\nFinancial impact for scenario 126: ${financial_impact_scenario_126:,.2f}")
print(f"Financial impact for scenario 245: ${financial_impact_scenario_245:,.2f}")
print(f"Financial impact for scenario 585: ${financial_impact_scenario_585:,.2f}")

average_financial_impact = (
    financial_impact_scenario_126
    + financial_impact_scenario_245
    + financial_impact_scenario_585
) / 3

print(f"\nAverage financial impact across scenarios: ${average_financial_impact:,.2f}")

## ESG Impact

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np

### a) Food Security Risk Mapping

We compare historical average yield (2010-2018) with predicted yield under the worst-case climate scenario (SSP5-8.5, 2040-2050) to identify the most vulnerable French departments.

In [ ]:
HISTORICAL_START, HISTORICAL_END = 2010, 2014
FUTURE_START, FUTURE_END = 2040, 2050

# Historical baseline: average yield per department (2010-2014)
hist_mask = historical_yield_data[col_silver.SILVER_YEAR].between(
    HISTORICAL_START, HISTORICAL_END
)
hist_yield = (
    historical_yield_data.loc[hist_mask]
    .groupby(col_silver.SILVER_NOM_DEP)[col_silver.SILVER_YIELD]
    .mean()
    .rename("hist_yield")
)

# Future prediction: average yield per department under SSP5-8.5 (2040-2050)
future_mask = scenario_585[col_dashboard.DASHBOARD_YEAR].between(
    FUTURE_START, FUTURE_END
)
future_yield = (
    scenario_585.loc[future_mask]
    .groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)[col_dashboard.DASHBOARD_YIELD]
    .mean()
    .rename("future_yield")
)

# Join and compute change
yield_change = hist_yield.to_frame().join(future_yield, how="inner")
yield_change["abs_change"] = yield_change["future_yield"] - yield_change["hist_yield"]
yield_change["pct_change"] = (
    yield_change["abs_change"] / yield_change["hist_yield"] * 100
)
yield_change = yield_change.sort_values("pct_change")

print(f"Departments with yield data: {len(yield_change)}")
print(f"Departments with projected decline: {(yield_change['pct_change'] < 0).sum()}")
print(f"Average yield change: {yield_change['pct_change'].mean():.1f}%")
print(
    f"Worst decline: "
    f"{yield_change['pct_change'].iloc[0]:.1f}% ({yield_change.index[0]})"
)

In [ ]:
TOP_N = 20

top_vulnerable = yield_change.head(TOP_N)

fig, ax = plt.subplots(figsize=(10, 8))
colors = plt.cm.RdYlGn(
    np.interp(
        top_vulnerable["pct_change"],
        [top_vulnerable["pct_change"].min(), 0],
        [0, 0.5],
    )
)
bars = ax.barh(
    range(len(top_vulnerable)),
    top_vulnerable["pct_change"],
    color=colors,
)
ax.set_yticks(range(len(top_vulnerable)))
ax.set_yticklabels(top_vulnerable.index)
ax.invert_yaxis()
ax.set_xlabel("Yield change (%)")
ax.set_title(
    f"Top {TOP_N} Most Vulnerable Departments\n"
    f"SSP5-8.5 ({FUTURE_START}-{FUTURE_END}) vs "
    f"Historical ({HISTORICAL_START}-{HISTORICAL_END})"
)
ax.axvline(x=0, color="black", linewidth=0.8)
ax.xaxis.set_major_formatter(mticker.PercentFormatter())

for bar, pct in zip(bars, top_vulnerable["pct_change"], strict=False):
    ax.text(
        bar.get_width() - 0.3,
        bar.get_y() + bar.get_height() / 2,
        f"{pct:.1f}%",
        va="center",
        ha="right",
        fontsize=8,
        fontweight="bold",
        color="white",
    )

plt.tight_layout()
plt.show()

### b) Land-Use / Carbon Implication

If yields decline but France wants to maintain the same barley production, more land must be cultivated. This land-use change (converting natural land to cropland) has a measurable carbon cost.

We use [IPCC](https://www.ipcc-nggip.iges.or.jp/public/2019rf/pdf/4_Volume4/19R_V4_Ch05_Cropland.pdf) estimates of **~100 tCO₂/ha** emitted when converting grassland/forest to cropland (one-time stock release).

In [ ]:
# IPCC estimate: CO2 released when converting grassland/forest to cropland
CO2_PER_HA_LAND_CONVERSION = 100

# Historical average area per department
hist_area = (
    historical_yield_data.loc[hist_mask]
    .groupby(col_silver.SILVER_NOM_DEP)[col_silver.SILVER_AREA]
    .mean()
    .rename("hist_area")
)

# Join area with yield change data
land_use = yield_change.join(hist_area, how="inner")

# Current production = historical yield * historical area
land_use["hist_production"] = land_use["hist_yield"] * land_use["hist_area"]

# Area needed to maintain same production at future yield
land_use["future_area_needed"] = land_use["hist_production"] / land_use["future_yield"]

# Extra land needed (positive = expansion required)
land_use["extra_land_ha"] = land_use["future_area_needed"] - land_use["hist_area"]

# National totals
total_extra_land = land_use["extra_land_ha"].sum()
total_co2 = total_extra_land * CO2_PER_HA_LAND_CONVERSION

print("=" * 60)
print("LAND-USE IMPACT TO MAINTAIN CURRENT PRODUCTION (SSP5-8.5)")
print("=" * 60)
print(f"Total historical cultivated area:  {land_use['hist_area'].sum():>12,.0f} ha")
print(
    f"Total area needed at future yield: "
    f"{land_use['future_area_needed'].sum():>12,.0f} ha"
)
print(f"Extra land required:               {total_extra_land:>12,.0f} ha")
print(f"  = {total_extra_land / 100:,.0f} km²")
print(
    f"Area increase:                     "
    f"{total_extra_land / land_use['hist_area'].sum() * 100:>11.1f} %"
)
print()
print("CARBON COST OF LAND-USE CHANGE")
print("-" * 60)
print(f"CO₂ from land conversion:          {total_co2:>12,.0f} tCO₂")
print(f"  = {total_co2 / 1e6:,.2f} MtCO₂")
print("  (at 100 tCO₂/ha, IPCC estimate for grassland → cropland)")

In [ ]:
# Top departments requiring the most extra land
top_land = land_use.nlargest(15, "extra_land_ha")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Left: extra land needed per department
colors1 = plt.cm.Oranges(
    np.interp(
        top_land["extra_land_ha"],
        [0, top_land["extra_land_ha"].max()],
        [0.3, 1.0],
    )
)
ax1.barh(range(len(top_land)), top_land["extra_land_ha"], color=colors1)
ax1.set_yticks(range(len(top_land)))
ax1.set_yticklabels(top_land.index)
ax1.invert_yaxis()
ax1.set_xlabel("Extra land needed (ha)")
ax1.set_title("Extra Cropland Needed to\nMaintain Production (SSP5-8.5)")

# Right: associated CO2 cost
co2_values = top_land["extra_land_ha"] * CO2_PER_HA_LAND_CONVERSION
colors2 = plt.cm.Reds(np.interp(co2_values, [0, co2_values.max()], [0.3, 1.0]))
ax2.barh(range(len(top_land)), co2_values / 1000, color=colors2)
ax2.set_yticks(range(len(top_land)))
ax2.set_yticklabels(top_land.index)
ax2.invert_yaxis()
ax2.set_xlabel("CO₂ from land conversion (ktCO₂)")
ax2.set_title("Carbon Cost of Land-Use Change\n(100 tCO₂/ha, IPCC estimate)")

plt.tight_layout()
plt.show()

### c) ESG Gain from Using the Model

**Without the model**: each department blindly expands cropland to compensate for its own yield loss.

**With the model**: we identify the 20 most vulnerable departments and reallocate their production to the 70 most resilient ones (same assumption as the financial impact section), where higher yields mean less land is needed per ton produced.

In [ ]:
N_DROPPED = 20  # same as financial impact section

total_production = land_use["hist_production"].sum()
total_hist_area = land_use["hist_area"].sum()

# --- WITHOUT model: every department expands independently (already computed) ---
without_model_area = land_use["future_area_needed"].sum()
without_model_extra = without_model_area - total_hist_area  # = 86,240 ha

# --- WITH model: reallocate to top departments ---
# Rank departments by future yield (best first)
land_use_ranked = land_use.sort_values("future_yield", ascending=False)

# Top 70 departments handle all production
top_deps = land_use_ranked.head(len(land_use_ranked) - N_DROPPED)

# Area these top departments need for their OWN historical production
top_own_area = (top_deps["hist_production"] / top_deps["future_yield"]).sum()

# Production from the dropped departments that must be absorbed
dropped_deps = land_use_ranked.tail(N_DROPPED)
redistributed_production = dropped_deps["hist_production"].sum()

# Extra area to absorb redistributed production (at the top departments' avg yield)
avg_yield_top = top_deps["future_yield"].mean()
redistributed_area = redistributed_production / avg_yield_top

# Total area with model = top departments' own needs + redistributed
with_model_area = top_own_area + redistributed_area

# The dropped departments free up their historical land (no longer cultivated)
freed_land = dropped_deps["hist_area"].sum()
with_model_extra = with_model_area - (total_hist_area - freed_land)

# --- GAIN ---
land_saved = without_model_extra - with_model_extra
co2_saved = land_saved * CO2_PER_HA_LAND_CONVERSION

print("=" * 60)
print("ESG GAIN FROM MODEL-DRIVEN REALLOCATION (SSP5-8.5)")
print("=" * 60)
print(f"Total national barley production:  {total_production:>12,.0f} t")
print()
print("WITHOUT model (blind expansion):")
print(f"  Extra land needed:               {without_model_extra:>12,.0f} ha")
print(
    f"  CO₂ cost:                        "
    f" {without_model_extra * CO2_PER_HA_LAND_CONVERSION:>12,.0f} tCO₂"
)
print()
print(f"WITH model (drop worst {N_DROPPED}, reallocate to top {len(top_deps)}):")
print(f"  Avg yield of top departments:    {avg_yield_top:>12.2f} t/ha")
print(f"  Land freed (abandoned deps):     {freed_land:>12,.0f} ha")
print(f"  Extra land needed:               {with_model_extra:>12,.0f} ha")
print(
    "  CO₂ cost:                        "
    f" {with_model_extra * CO2_PER_HA_LAND_CONVERSION:>12,.0f} tCO₂"
)
print()
print("GAIN FROM USING THE MODEL")
print("-" * 60)
print(
    f"  Land conversion avoided:         "
    f"{land_saved:>12,.0f} ha ({land_saved / 100:,.0f} km²)"
)
print(f"  CO₂ saved:                       {co2_saved:>12,.0f} tCO₂")
print(f"    = {co2_saved / 1e6:,.2f} MtCO₂")
print(f"    = {co2_saved / total_co2 * 100:.0f}% of the naive-scenario carbon cost")